# Setup

## Import

In [ ]:
from brian2 import *
from scipy.signal import savgol_filter
import pickle

In [ ]:
import os.path
from os import path

## Saving
total_save_seconds = 180
save_path_leafs = ['60','120','180','240']
save_path_prefix = './out/long_run/'
##save_path_prefix = './out/test_run/'

if path.exists(save_path_prefix) == False:
    os.mkdir(save_path_prefix)

for p in range(60, total_save_seconds+60, 60):
  if path.exists(save_path_prefix + str(p)) == False:
    os.mkdir(save_path_prefix + str(p))

os.chdir(save_path_prefix)
!pwd
!ls


In [ ]:
!pwd

## Defs

### LIF Song, Miller and Abbot neuron
    dv/dt = (ge * (Ee-v) + El - v) / taum : volt

### Simple synapse
    on_pre='v += s'

### STDP synapse
    on_pre='''ge += s


### Izhi neuron
    dv/dt = (0.04*v**2 + 5*v + 140 - u + I + I_noise )/ms : 1


### Izhi simple synapse
    on_pre={"up": "I += w", "down": "I -= w"}
    
### Izhi STDP synapse
    on_pre='''I += s


### Functions 

### Izhi parameters
![](https://raw.githubusercontent.com/max-talanov/1/master/bypass/Izhi_parameters.png)

In [ ]:

def generate_LIF_neurons(neurons_n):
    neurons = NeuronGroup(neurons_n, 
                        #'''dv/dt = (ge * (Ee-v) + El - v) / taum : volt
                        '''dv/dt = (ge * (Ee-v) + El - v) / taum : volt
                        dge/dt = -ge / taue : 1''',
                        threshold='v>vt', reset='v = vr',
                        # method='exact'
                        method='euler')
    return neurons

def generate_Izhi_neurons(neurons_n, a=0.02, b=0.2, c=-65., d=8):
    '''Generates regural spiking neurons (RS)'''
    eqs = """dv/dt = (0.04*v**2 + 5*v + 140 - u + I + I_noise )/ms : 1
         du/dt = (a*(b*v - u))/ms  : 1
         I : 1
         I_noise : 1
         a : 1
         b : 1
         cn : 1
         dn : 1
       """
    N = NeuronGroup(neurons_n, eqs, threshold="v>=30", reset="v=cn; u+=dn", method="euler")
    N.v = -65
    N.a = a
    N.b = b
    N.cn = c
    N.dn = d
    return N

def generate_Izhi_chattering_neurons(neurons_n):
    res = generate_Izhi_neurons(neurons_n, a=0.02, b=0.2, c=-50, d=2)
    return res

def generate_Izhi_RS_neurons(neurons_n):
    res = generate_Izhi_neurons(neurons_n)
    return res

def generate_STDP_DA_synapses(pre, post):
    """
    Generates the STDP modulated with DA synapses
    pre -- the presynaptic neurons 
    post -- the postsynaptic neurons 
    mode is external variable: mode = 0 switches off DA modulation, mode = 1 switches on DA modulation.
    s -- synaptic strength      = 1e-10
    c -- eligibility trace      = 1e-10
    d -- extracellular dopamine = 0
    """
    synapse_stdp = Synapses(pre, post,
                          model='''mode: 1
                          dc/dt = -c / tauc : 1 (clock-driven)
                          dd/dt = -d / taud : 1 (clock-driven)
                          ds/dt = mode * c * d / taus : 1 (clock-driven)
                          dApre/dt = -Apre / taupre : 1 (event-driven)
                          dApost/dt = -Apost / taupost : 1 (event-driven)''',
                          on_pre='''v += s
                          Apre += dApre
                          c = clip(c + mode * Apost, -gmax, gmax)
                          s = clip(s + (1-mode) * Apost, -gmax, gmax)
                          ''',
                          on_post='''Apost += dApost
                          c = clip(c + mode * Apre, -gmax, gmax)
                          s = clip(s + (1-mode) * Apre, -gmax, gmax)
                          ''',
                          method='euler'
                          )
    return synapse_stdp

def generate_ex_STDP_DA_synapses(pre, post):
  """
    Generates the STDP modulated with DA synapses
    pre -- the presynaptic neurons 
    post -- the postsynaptic neurons 
    mode is external variable: mode = 0 switches off DA modulation, mode = 1 switches on DA modulation.
    """
  synapse_stdp = Synapses(pre, post,
                          model='''mode: 1
                          dc/dt = -c / tauc : 1 (clock-driven)
                          dd/dt = -d / taud : 1 (clock-driven)
                          ds/dt = mode * c * d / taus : 1 (clock-driven)
                          dApre/dt = -Apre / taupre : 1 (event-driven)
                          dApost/dt = -Apost / taupost : 1 (event-driven)''',
                          on_pre='''v += s
                          Apre += dApre
                          c = clip(c + mode * Apost, -gmax, gmax)
                          s = clip(s + (1-mode) * Apost, 0, gmax)
                          ''',
                          on_post='''Apost += dApost
                          c = clip(c + mode * Apre, -gmax, gmax)
                          s = clip(s + (1-mode) * Apre, 0, gmax)
                          ''',
                          method='euler'
                          )
  return synapse_stdp

def generate_inh_STDP_DA_synapses(pre, post):
  """
    Generates the STDP modulated with DA synapses
    pre -- the presynaptic neurons 
    post -- the postsynaptic neurons 
    mode is external variable: mode = 0 switches off DA modulation, mode = 1 switches on DA modulation.
    """
  synapse_stdp = Synapses(pre, post,
                          model='''mode: 1
                          dc/dt = -c / tauc : 1 (clock-driven)
                          dd/dt = -d / taud : 1 (clock-driven)
                          ds/dt = mode * c * d / taus : 1 (clock-driven)
                          dApre/dt = -Apre / taupre : 1 (event-driven)
                          dApost/dt = -Apost / taupost : 1 (event-driven)''',
                          on_pre='''v += s
                          Apre += dApre
                          c = clip(c + mode * Apost, -gmax, gmax)
                          s = clip(s + (1-mode) * Apost, -gmax, 0)
                          ''',
                          on_post='''Apost += dApost
                          c = clip(c + mode * Apre, -gmax, gmax)
                          s = clip(s + (1-mode) * Apre, -gmax, 0)
                          ''',
                          method='euler'
                          )
  return synapse_stdp

def generate_simple_synapses(pre, post):
  synapse = Synapses(pre, post, model='''s: 1''', on_pre='v += s')
  return synapse

def identity_cut_chank(step):
  ## cut_chank_number
  cut_chank_number = [0]
  if (step < 2):   cut_chank_number = [0]
  elif (step < 3): cut_chank_number = [0,1]
  elif (step < 4): cut_chank_number = [1]
  elif (step < 6): cut_chank_number = [1,2]
  elif (step < 7): cut_chank_number = [2]
  elif (step < 8): cut_chank_number = [2,3]
  elif (step < 10): cut_chank_number = [3]
  return cut_chank_number

def identity_cut_chank_no_intersections(step):
  ## cut_chank_number
  cut_chank_number = [0]
  if (step < 2):    cut_chank_number = [0]
  elif (step < 3):  cut_chank_number = [0]
  elif (step < 4):  cut_chank_number = [1]
  elif (step < 6):  cut_chank_number = [1]
  elif (step < 7):  cut_chank_number = [2]
  elif (step < 8):  cut_chank_number = [2]
  elif (step < 10): cut_chank_number = [3]
  return cut_chank_number

def identify_muscle_freq(step, freq_hi, freq_lo):
  if step in [2,3,6,7]: return freq_hi
  else: return freq_lo

def identify_V3_F_freq(step, freq_hi, freq_lo):
    ## [6,7]
    if step in [6,7]: return freq_hi
    else: return freq_lo

def identify_V0_freq(step, freq_hi, freq_lo):
    ## [6,7]
    if step in [6,7]: return freq_hi
    else: return freq_lo

def identify_bs_freq(step, freq_hi, freq_lo, steps_num):
    freq_step = (freq_hi-freq_lo)/steps_num
    return step*freq_step + freq_lo

def save_reset_spike_monitor(spike_monitor, save_path_prefix, file_name, neuron_group):
    print ('saving spike montor' + str(spike_monitor) + ' into: ' + save_path_prefix)
    data = spike_monitor.get_states(['t', 'i'])
    save_path_run = save_path_prefix + '/'
    print('save in ' + save_path_run + file_name + '.pickle')
    with open(save_path_run + file_name + '.pickle', 'wb') as f:
        pickle.dump(data, f)
        print('reset monitor')
        del spike_monitor
        del data
        print('creating new monitor')
        res_monitor = SpikeMonitor(neuron_group)
        return res_monitor

def save_reset_state_monitor(state_monitor, save_path_prefix, file_name, state, monitor_dt, group):
    print ('saving state montor' + str(state_monitor))
    data = state_monitor.get_states(['t', state])
    save_path_run = save_path_prefix + '/'
    print('save '+ str(state) + ' in ' + save_path_run + file_name + '.pickle')
    with open(save_path_run + file_name + '.pickle', 'wb') as f:
        pickle.dump(data, f)
        print('reset monitor')
        del state_monitor
        del data
        print('creating new monitor')
        res_monitor = StateMonitor(group, state, record=True, dt=monitor_dt)
        return res_monitor

print('Completed functions definitions ...')

### Parameters: 10 steps

In [ ]:
# Steps
steps_num = 20 # 120

step_duration = 1000 ## ms
swing_stance_duration = 500 ##ms
simulation_hill_toe_phases = 4
simulation_muscle_phases = 5
simulation_step_phases = 10

# Inputs F&E connections not STDP
cut2rg_conn_probability = 1.0 # 0.1 # 0.02 # 0.01 ## 0.02 ## cut -> rg; 🔼🔼
muscle2rg_conn_probability = 0.2 # 0.01 #0.1 # 0.02 # 0.1 ## 0.02 ## muscle -> rg; 
V3_F_2rg_conn_probability = 0.3 # 0.1 ## V3-F -> rg

InX2X_conn_probability = 0.1 # 0.05 # E->InE->F & F->InF->E;
rg2motor_conn_probability = 1.0 # 0.3 # 0.1 ## rg -> motor 🔼🔼

## 2Ia
#cut2Ia_conn_probability = 0.1 ## cut -> Ia
muscle2Ia_conn_probability = 0.1 ## muscle -> Ia
Ia2motor_conn_probability = 0.1 

# Commissural projections
V0d_conn_probability = 0.2
V2a_conn_probability = 0.2
V0v_conn_probability = 0.2
In1_conn_probability = 0.1
InX_conn_probability = 0.1
bs_e2rg_conn_probability = 0.10 # 0.05
bs_f2rg_conn_probability = 0.01 # 0.05

# STDP
cut_stdp_synapse_probability = 0.9 # 0.01 # 0.9 ## 0.1 ## cut -> rg 🔼🔼
muscle_stdp_synapse_probability = 0.01 # 0.9 # 0.9 ## 0.1 ## cut -> rg
InX_stdp_synapse_probability = 0.1 # 0.9 ## E->InE->F & F->InF->E
default_stdp_mode = 0
default_stdp_s = 1e-10 
default_stdp_c = 1e-10
default_stdp_d = 0
cut_stdp_mode = default_stdp_mode
cut_stdp_s = 1e-10 # 1.0 # 1e-3 # 1e-10 
cut_stdp_c = 1e-10 # 1.0 # 1e-3 # 1e-10 
cut_stdp_d = default_stdp_d 

# Weights 
default_weight = 20. # 100. # 10. ##100.
cut_weight = 40. # 50. # 70. # 50 # 70. # 100. # 
bs_weight = 10. ## 30.
motor_weight = 100. # 100.0 

# Nubers and frequencies
## Cut fibers 
cut_fibers_num = 100 ## 24 ## 100
cut_fibers_freq = 50*Hz #200*Hz

## Brain stem
bs_e_fibers_num = 100
bs_e_fibers_freq = 1.0*Hz
bs_f_fibers_num = 100
bs_f_fibers_freq_lo = 0.1*Hz
bs_f_fibers_freq_hi = 5*Hz

## Muscles
muscle_fibers_num = 100 # 8 #100
# [20,50,20,50,20]*Hz
muscle_fibers_freq_hi = 50*Hz
muscle_fibers_freq_lo = 20*Hz

## V3
V3_F_num = 200
V3_F_freq_lo = 5*Hz 
V3_F_freq_hi = 200*Hz 

V0v_num = 100
V2a_num = 100
In1_num = 100
V0d_num = 100
InX_num = 100
Ia_num = 100
rg_num = 200 # 48 # 200
motor_num = 200 # 48 # 200

## Neurons
taum = 10*ms
Ee = 0*mV
vt = -54*mV
vr = -60*mV
El = -74*mV
taue = 5*ms

## STDP
taupre = 20*ms
taupost = taupre
## gmax = 10 
gmax = 70. # 30.0 # .01
## dApre = 1 
dApre = 0.01 # 0.1 # 0.01 # 1.0 # .01 # .01 🔽
dApost = -dApre * taupre / taupost * 1.05
dApost *= gmax
dApre *= gmax
#Monitors 
stdp_monitor_dt= 20*ms ##5*ms
delay_max = 3*ms

## Dopamine signaling
tauc = 1000*ms
taud = 200*ms
taus = 1*ms
epsilon_dopa = 5e-3

print('Parameters set ...')

## Network 2 legs

![](https://raw.githubusercontent.com/max-talanov/1/master/bypass/CPG_feedback_loops.png)


### Left leg

#### Extensor

In [6]:
# Setting the stage
start_scope()

# Extensor
# Inputs
l_e_poisson_cut_input = PoissonGroup(cut_fibers_num, rates=cut_fibers_freq)
l_e_poisson_muscle_input = PoissonGroup(muscle_fibers_num, rates=muscle_fibers_freq_lo)
# Rythm generators
l_e_rg_neurons = generate_Izhi_neurons(rg_num)
#l_e_rg_neurons.v = vr
# Motor neurons
l_e_motor_neurons = generate_Izhi_neurons(motor_num)
#l_e_motor_neurons.v = vr
## synapse = generate_simple_synapses(spike_input, neurons)
l_e_cut2rg = generate_simple_synapses(l_e_poisson_cut_input, l_e_rg_neurons)
l_e_cut2rg.connect(condition='j==2*i or j==(2*i+1)', p=cut2rg_conn_probability)
l_e_cut2rg.s = cut_weight

## synapse = generate_simple_synapses(spike_input, neurons)
l_e_muscle2rg = generate_simple_synapses(l_e_poisson_muscle_input, l_e_rg_neurons)
l_e_muscle2rg.connect(p=muscle2rg_conn_probability)
## l_e_muscle2rg.s = 100. * mV
l_e_muscle2rg.s = default_weight

l_e_rg2motor = generate_simple_synapses(l_e_rg_neurons, l_e_motor_neurons)
l_e_rg2motor.connect(condition='j==i', p=rg2motor_conn_probability)
## l_e_rg2motor.s = 100. * mV
l_e_rg2motor.s = motor_weight 

## cut stdp synapses
l_e_cut2rg_stdp = generate_ex_STDP_DA_synapses(l_e_poisson_cut_input, l_e_rg_neurons)
l_e_cut2rg_stdp.connect(condition='j==2*i or j==(2*i+1)', p=cut_stdp_synapse_probability)
l_e_cut2rg_stdp.mode = cut_stdp_mode
l_e_cut2rg_stdp.s = cut_stdp_s
l_e_cut2rg_stdp.c = cut_stdp_c
l_e_cut2rg_stdp.d = cut_stdp_d
# e_cut2rg_stdp.s = 'rand() * 2 * gmax - gmax'
l_e_cut2rg_stdp.pre.delay = 'rand() * delay_max'

##muscle stdp synapses
l_e_muscle2rg_stdp = generate_ex_STDP_DA_synapses(l_e_poisson_muscle_input, l_e_rg_neurons)
l_e_muscle2rg_stdp.connect(p=muscle_stdp_synapse_probability)
l_e_muscle2rg_stdp.mode = default_stdp_mode
l_e_muscle2rg_stdp.s = default_stdp_s
l_e_muscle2rg_stdp.c = default_stdp_c
l_e_muscle2rg_stdp.d = default_stdp_d
# l_e_muscle2rg_stdp.s = 'rand() * 2 * gmax - gmax'
l_e_muscle2rg_stdp.pre.delay = 'rand() * delay_max'

l_e_muscle2rg_stdp

Synapses(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='synapses_4')

#### Flexor

#### Brain stem

#### F↔E projections

### Right leg

#### Extensor

In [7]:
# Extensor
# Inputs
r_e_poisson_cut_input = PoissonGroup(cut_fibers_num, rates=cut_fibers_freq)
r_e_poisson_muscle_input = PoissonGroup(muscle_fibers_num, rates=muscle_fibers_freq_lo)
# Rythm generators
r_e_rg_neurons = generate_Izhi_neurons(rg_num)
#r_e_rg_neurons.v = vr
# Motor neurons
r_e_motor_neurons = generate_Izhi_neurons(motor_num)
#r_e_motor_neurons.v = vr
## synapse = generate_simple_synapses(spike_input, neurons)
r_e_cut2rg = generate_simple_synapses(r_e_poisson_cut_input, r_e_rg_neurons)
r_e_cut2rg.connect(condition='j==2*i or j==(2*i+1)', p=cut2rg_conn_probability)
r_e_cut2rg.s = cut_weight 

## synapse = generate_simple_synapses(spike_input, neurons)
r_e_muscle2rg = generate_simple_synapses(r_e_poisson_muscle_input, r_e_rg_neurons)
r_e_muscle2rg.connect(p=muscle2rg_conn_probability)
r_e_muscle2rg.s = default_weight

r_e_rg2motor = generate_simple_synapses(r_e_rg_neurons, r_e_motor_neurons)
## r_e_rg2motor.connect(p=rg2motor_conn_probability) 
r_e_rg2motor.connect(condition='j==i', p=rg2motor_conn_probability)
r_e_rg2motor.s = motor_weight

r_e_cut2rg_stdp = generate_ex_STDP_DA_synapses(r_e_poisson_cut_input, r_e_rg_neurons)
r_e_cut2rg_stdp.connect(condition='j==2*i or j==(2*i+1)',p=cut_stdp_synapse_probability)
r_e_cut2rg_stdp.mode = cut_stdp_mode
r_e_cut2rg_stdp.s = cut_stdp_s
r_e_cut2rg_stdp.c = cut_stdp_c
r_e_cut2rg_stdp.d = cut_stdp_d
# r_e_cut2rg_stdp.s = 'rand() * 2 * gmax - gmax'
r_e_cut2rg_stdp.pre.delay = 'rand() * delay_max'

##muscle stdp synapses
r_e_muscle2rg_stdp = generate_ex_STDP_DA_synapses(r_e_poisson_muscle_input, r_e_rg_neurons)
r_e_muscle2rg_stdp.connect(p=muscle_stdp_synapse_probability)
r_e_muscle2rg_stdp.mode = default_stdp_mode
r_e_muscle2rg_stdp.s = default_stdp_s
r_e_muscle2rg_stdp.c = default_stdp_c
r_e_muscle2rg_stdp.d = default_stdp_d
# r_e_muscle2rg_stdp.s = 'rand() * 2 * gmax - gmax'
r_e_muscle2rg_stdp.pre.delay = 'rand() * delay_max'

r_e_muscle2rg_stdp

Synapses(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='synapses_9')

#### Flexor

#### Brainstem

#### F↔E projections

### left↔right coordination 

## Monitors

In [8]:
# Left leg
## Extensor
l_e_cut_monitor = SpikeMonitor(l_e_poisson_cut_input)
l_e_muscle_monitor = SpikeMonitor(l_e_poisson_muscle_input)
#l_e_bs_monitor = SpikeMonitor(l_e_poisson_bs_input)
l_e_rg_neurons_monitor = SpikeMonitor(l_e_rg_neurons)
l_e_motor_neurons_monitor = SpikeMonitor(l_e_motor_neurons)
#l_e_Ia_neurons_monitor = SpikeMonitor(l_e_Ia_neurons)
## Voltages
l_e_motor_neurons_voltages = StateMonitor(l_e_motor_neurons, 'v', record=True, dt=stdp_monitor_dt)
l_e_cut2rg_stdp_monitor = StateMonitor(l_e_cut2rg_stdp, 's', record=True, dt=stdp_monitor_dt)
l_e_muscle2rg_stdp_monitor = StateMonitor(l_e_muscle2rg_stdp, 's', record=True, dt=stdp_monitor_dt)
#l_e_rg2InE_stdp_monitor = StateMonitor(l_e_rg2InE_stdp, 's', record=True, dt=stdp_monitor_dt)
## Flexor
# l_f_muscle_monitor = SpikeMonitor(l_f_poisson_muscle_input)
# l_f_V3_F_monitor = SpikeMonitor(l_f_poisson_V3_F_input)
# l_f_rg_neurons_monitor = SpikeMonitor(l_f_rg_neurons)
# l_f_motor_neurons_monitor = SpikeMonitor(l_f_motor_neurons)
# l_f_Ia_neurons_monitor = SpikeMonitor(l_f_Ia_neurons)
# l_f_V0d_neurons_monitor = SpikeMonitor(l_f_V0d_neurons)
# l_f_In1_neurons_monitor = SpikeMonitor(l_f_In1_neurons)
# l_f_bs_monitor = SpikeMonitor(l_f_poisson_bs_input)
## Voltages
# l_f_motor_neurons_voltages = StateMonitor(l_f_motor_neurons, 'v', record=True, dt=stdp_monitor_dt)
# l_f_muscle2rg_stdp_monitor = StateMonitor(l_f_muscle2rg_stdp, 's', record=True, dt=stdp_monitor_dt)
# l_f_rg2InE_stdp_monitor = StateMonitor(l_f_rg2InE_stdp, 's', record=True, dt=stdp_monitor_dt)
# Right leg
## Extensor
r_e_cut_monitor = SpikeMonitor(r_e_poisson_cut_input)
r_e_muscle_monitor = SpikeMonitor(r_e_poisson_muscle_input)
#r_e_bs_monitor = SpikeMonitor(r_e_poisson_bs_input)
r_e_rg_neurons_monitor = SpikeMonitor(r_e_rg_neurons)
r_e_motor_neurons_monitor = SpikeMonitor(r_e_motor_neurons)
#r_e_Ia_neurons_monitor = SpikeMonitor(r_e_Ia_neurons)

## Voltages
r_e_motor_neurons_voltages = StateMonitor(r_e_motor_neurons, 'v', record=True, dt=stdp_monitor_dt)
r_e_cut2rg_stdp_monitor = StateMonitor(r_e_cut2rg_stdp, 's', record=True, dt=stdp_monitor_dt)
r_e_muscle2rg_stdp_monitor = StateMonitor(r_e_muscle2rg_stdp, 's', record=True, dt=stdp_monitor_dt)
#r_e_rg2InE_stdp_monitor = StateMonitor(r_e_rg2InE_stdp, 's', record=True, dt=stdp_monitor_dt)

## Flexor
# r_f_muscle_monitor = SpikeMonitor(r_f_poisson_muscle_input)
# r_f_V3_F_monitor = SpikeMonitor(r_f_poisson_V3_F_input)
# r_f_rg_neurons_monitor = SpikeMonitor(r_f_rg_neurons)
# r_f_motor_neurons_monitor = SpikeMonitor(r_f_motor_neurons)
# r_f_Ia_neurons_monitor = SpikeMonitor(r_f_Ia_neurons)
# r_f_V0d_neurons_monitor = SpikeMonitor(r_f_V0d_neurons)
# r_f_In1_neurons_monitor = SpikeMonitor(r_f_In1_neurons)
# r_f_bs_monitor = SpikeMonitor(r_f_poisson_bs_input)

## Voltages
# r_f_motor_neurons_voltages = StateMonitor(r_f_motor_neurons, 'v', record=True, dt=stdp_monitor_dt)
# r_f_muscle2rg_stdp_monitor = StateMonitor(r_f_muscle2rg_stdp, 's', record=True, dt=stdp_monitor_dt)
# r_f_rg2InE_stdp_monitor = StateMonitor(r_f_rg2InF_stdp, 's', record=True, dt=stdp_monitor_dt)

print('Monitor setup complete ...')

Monitor setup complete ...


# Run 
![](https://raw.githubusercontent.com/max-talanov/1/master/bypass/step_phases.png)

In [9]:
!pwd

/home/talanov/brian2/bypass/out/long_run


In [10]:
# Simulation
## Classical STDP
#synapse_stdp.mode = 0
## Split simulation in N chanks and switch on sequentally 
dur = int(swing_stance_duration/simulation_step_phases)
cut_chank = int(len(l_e_poisson_cut_input)/simulation_hill_toe_phases)
print('Simulation started ...')
## print (dur)
## print (cut_chank)
steps_to_save = 60
save_path_prefix = './'

for step in range(steps_num): 
    print ('step=' + str(step))
    ## TODO update into swing and stance
    if ((step % steps_to_save) == 0) and step > 0:
        save_path_run = save_path_prefix + str(step)
        # Left leg
        ## Extensor 
        # Save states
        l_e_synapse_stdp_monitor=save_reset_state_monitor(l_e_cut2rg_stdp_monitor, save_path_run, 
                                                        'l_e_cut2rg_weigts', 's', 
                                                  stdp_monitor_dt, l_e_cut2rg_stdp)
        l_e_motor_neurons_voltages=save_reset_state_monitor(l_e_motor_neurons_voltages, save_path_run, 
                                                    'l_e_motor_neurons_voltages', 'v', 
                                                    stdp_monitor_dt, l_e_motor_neurons)
        # Save spikes
        l_e_motor_neurons_monitor=save_reset_spike_monitor(l_e_motor_neurons_monitor, save_path_run, 
                                                   'l_e_motor_neurons_spikes', l_e_motor_neurons)
        l_e_rg_neurons_monitor=save_reset_spike_monitor(l_e_rg_neurons_monitor, save_path_run, 
                                                   'l_e_rg_neurons_spikes', l_e_rg_neurons)
        l_e_cut_monitor=save_reset_spike_monitor(l_e_cut_monitor, save_path_run, 
                                                   'l_e_cut_spikes', l_e_poisson_cut_input)
        l_e_muscle_monitor=save_reset_spike_monitor(l_e_muscle_monitor, save_path_run, 
                                                   'l_e_muscle_spikes', l_e_poisson_muscle_input)
#        l_e_Ia_neurons_monitor=save_reset_spike_monitor(l_e_Ia_neurons_monitor, save_path_run, 
#                                                   'l_e_Ia_spikes', l_e_Ia_neurons)
  
        ## Flexor 
        # Save states
#         l_f_synapse_stdp_monitor=save_reset_state_monitor(l_f_muscle2rg_stdp_monitor, save_path_run, 
#                                                         'l_f_muscle2rg_weigts', 's', 
#                                                         stdp_monitor_dt, l_f_muscle2rg_stdp)
#         l_f_motor_neurons_voltages=save_reset_state_monitor(l_f_motor_neurons_voltages, save_path_run, 
#                                                           'l_f_motor_neurons_voltages', 'v', 
#                                                           stdp_monitor_dt, l_f_motor_neurons)
#         # Save spikes
#         l_f_motor_neurons_monitor=save_reset_spike_monitor(l_f_motor_neurons_monitor, save_path_run, 
#                                                    'l_f_motor_neurons_spikes', l_f_motor_neurons)
#         l_f_rg_neurons_monitor=save_reset_spike_monitor(l_f_rg_neurons_monitor, save_path_run, 
#                                                    'l_f_rg_neurons_spikes', l_f_rg_neurons)
#         l_f_muscle_monitor=save_reset_spike_monitor(l_f_muscle_monitor, save_path_run, 
#                                                    'l_f_muscle_spikes', l_f_poisson_muscle_input)
#         l_f_V3_F_monitor=save_reset_spike_monitor(l_f_V3_F_monitor, save_path_run, 
#                                                    'l_f_V3_F_spikes', l_f_poisson_V3_F_input)
#         l_f_Ia_neurons_monitor=save_reset_spike_monitor(l_f_Ia_neurons_monitor, save_path_run, 
#                                                    'l_f_Ia_spikes', l_f_Ia_neurons)
#         l_f_V0d_neurons_monitor=save_reset_spike_monitor(l_f_V0d_neurons_monitor, save_path_run, 
#                                                    'l_f_V0d_spikes', l_f_V0d_neurons)
#         l_f_In1_neurons_monitor=save_reset_spike_monitor(l_f_In1_neurons_monitor, save_path_run, 
#                                                    'l_f_In1_spikes', l_f_In1_neurons)
        
        # Right leg
        ## Extensor 
        # Save states
        r_e_synapse_stdp_monitor=save_reset_state_monitor(r_e_cut2rg_stdp_monitor, save_path_run, 
                                                        'r_e_cut2rg_weigts', 's', 
                                                  stdp_monitor_dt, r_e_cut2rg_stdp)
        r_e_motor_neurons_voltages=save_reset_state_monitor(r_e_motor_neurons_voltages, save_path_run, 
                                                    'r_e_motor_neurons_voltages', 'v', 
                                                    stdp_monitor_dt, r_e_motor_neurons)
        # Save spikes
        r_e_motor_neurons_monitor=save_reset_spike_monitor(r_e_motor_neurons_monitor, save_path_run, 
                                                   'r_e_motor_neurons_spikes', r_e_motor_neurons)
        r_e_rg_neurons_monitor=save_reset_spike_monitor(r_e_rg_neurons_monitor, save_path_run, 
                                                   'r_e_rg_neurons_spikes', r_e_rg_neurons)
        r_e_cut_monitor=save_reset_spike_monitor(r_e_cut_monitor, save_path_run, 
                                                   'r_e_cut_spikes', r_e_poisson_cut_input)
        r_e_muscle_monitor=save_reset_spike_monitor(r_e_muscle_monitor, save_path_run, 
                                                   'r_e_muscle_spikes', r_e_poisson_muscle_input)
#        r_e_Ia_neurons_monitor=save_reset_spike_monitor(r_e_Ia_neurons_monitor, save_path_run, 
#                                                   'r_e_Ia_spikes', r_e_Ia_neurons)  
        ## Flexor 
        # Save states
#         r_f_synapse_stdp_monitor=save_reset_state_monitor(r_f_muscle2rg_stdp_monitor, save_path_run, 
#                                                         'r_f_muscle2rg_weigts', 's', 
#                                                         stdp_monitor_dt, r_f_muscle2rg_stdp)
#         r_f_motor_neurons_voltages=save_reset_state_monitor(r_f_motor_neurons_voltages, save_path_run, 
#                                                           'r_f_motor_neurons_voltages', 'v', 
#                                                           stdp_monitor_dt, r_f_motor_neurons)
#         # Save spikes
#         r_f_motor_neurons_monitor=save_reset_spike_monitor(r_f_motor_neurons_monitor, save_path_run, 
#                                                    'r_f_motor_neurons_spikes', r_f_motor_neurons)
#         r_f_rg_neurons_monitor=save_reset_spike_monitor(r_f_rg_neurons_monitor, save_path_run, 
#                                                    'r_f_rg_neurons_spikes', r_f_rg_neurons)
#         r_f_muscle_monitor=save_reset_spike_monitor(r_f_muscle_monitor, save_path_run, 
#                                                    'r_f_muscle_spikes', r_f_poisson_muscle_input)
#         r_f_V3_F_monitor=save_reset_spike_monitor(l_f_V3_F_monitor, save_path_run, 
#                                                    'r_f_V3_F_spikes', r_f_poisson_V3_F_input)
#         r_f_Ia_neurons_monitor=save_reset_spike_monitor(l_f_Ia_neurons_monitor, save_path_run, 
#                                                    'r_f_Ia_spikes', r_f_Ia_neurons)
#         r_f_V0d_neurons_monitor = save_reset_spike_monitor(r_f_V0d_neurons_monitor, save_path_run, 
#                                                    'r_f_V0d_spikes', r_f_V0d_neurons)
#         r_f_In1_neurons_monitor = save_reset_spike_monitor(r_f_In1_neurons_monitor, save_path_run, 
#                                                    'r_f_In1_spikes', r_f_In1_neurons)

    swing_stance_phase = 0
    ## Left leg: Stance phase
    ## Right leg: Swing phase
    l_e_poisson_cut_input.rates = 0*Hz
    l_e_poisson_muscle_input.rates = 0*Hz
#     l_f_poisson_muscle_input.rates = 0*Hz
#     l_f_poisson_V3_F_input.rates = 0*Hz
#     l_f_poisson_bs_input.rates = 0*Hz
    r_e_poisson_cut_input.rates = 0*Hz
    r_e_poisson_muscle_input.rates = 0*Hz    
#     r_f_poisson_muscle_input.rates = 0*Hz
#     r_f_poisson_V3_F_input.rates = 0*Hz
#     r_f_poisson_bs_input.rates = 0*Hz
    for phase in range(simulation_step_phases):
        ### Left leg
        ## print ('stance phase=' + str(phase))
        l_e_cut_chank_number = identity_cut_chank(phase)
        # e_cut_chank_number = identity_cut_chank_no_intersections(phase)
        l_e_muscle_chank_freq = identify_muscle_freq(phase, muscle_fibers_freq_hi, muscle_fibers_freq_lo)
        ## print (cut_chank_number)
        ## print (muscle_chank_freq)
        l_e_poisson_cut_input[:].rates = 0*Hz
        for chn in l_e_cut_chank_number:
            ## print (str(chn*cut_chank) + ": " + str((chn+1)*cut_chank-1))
            l_e_poisson_cut_input[chn*cut_chank:(chn+1)*cut_chank-1].rates = 200*Hz
            ## print (poisson_cut_input)
        l_e_poisson_muscle_input.rates = l_e_muscle_chank_freq
        ### Right leg
        ## print ('swing phase=' + str(phase))
#         r_f_muscle_chank_freq = identify_muscle_freq(phase, muscle_fibers_freq_hi, muscle_fibers_freq_lo)
#         r_f_V3_F_chank_freq = identify_V3_F_freq(phase, V3_F_freq_hi, V3_F_freq_lo)
#         ## print (muscle_chank_freq)
#         r_f_poisson_muscle_input.rates = r_f_muscle_chank_freq
#         r_f_poisson_V3_F_input.rates = r_f_V3_F_chank_freq
#         r_f_poisson_bs_input.rates = identify_bs_freq(swing_stance_phase*10+phase, 
#                                                       bs_f_fibers_freq_hi, bs_f_fibers_freq_lo, 
#                                                       simulation_step_phases*2)
#         l_f_poisson_bs_input.rates = identify_bs_freq(swing_stance_phase*10+phase, 
#                                                       bs_f_fibers_freq_hi, bs_f_fibers_freq_lo, 
#                                                       simulation_step_phases*2)
        
        run(dur*ms)
        ## run(dur*ms, report='text')

    swing_stance_phase = 1
    ## Left leg: Swing phase
    ## Rigt leg: Stance phase 
    l_e_poisson_cut_input.rates = 0*Hz
    l_e_poisson_muscle_input.rates = 0*Hz
#     l_f_poisson_muscle_input.rates = 0*Hz
#     l_f_poisson_V3_F_input.rates = 0*Hz
#     ## l_f_poisson_bs_input.rates = 0*Hz
#     r_f_poisson_muscle_input.rates = 0*Hz
#     r_f_poisson_V3_F_input.rates = 0*Hz
#     ## r_f_poisson_bs_input.rates = 0*Hz
    r_e_poisson_cut_input.rates = 0*Hz
    r_e_poisson_muscle_input.rates = 0*Hz
    for phase in range(simulation_step_phases):
        ### Left leg
        ## print ('swing phase=' + str(phase))
#         l_f_muscle_chank_freq = identify_muscle_freq(phase, muscle_fibers_freq_hi, muscle_fibers_freq_lo)
#         l_f_V3_F_chank_freq = identify_V3_F_freq(phase, V3_F_freq_hi, V3_F_freq_lo)
#         ## print (muscle_chank_freq)
#         l_f_poisson_muscle_input.rates = l_f_muscle_chank_freq
#         l_f_poisson_V3_F_input.rates = l_f_V3_F_chank_freq
#         r_f_poisson_bs_input.rates = identify_bs_freq(swing_stance_phase*10+phase, 
#                                                       bs_f_fibers_freq_hi, bs_f_fibers_freq_lo, 
#                                                       simulation_step_phases*2)
#         l_f_poisson_bs_input.rates = identify_bs_freq(swing_stance_phase*10+phase, 
#                                                       bs_f_fibers_freq_hi, bs_f_fibers_freq_lo, 
#                                                       simulation_step_phases*2)
        ### Right leg
        ## print ('stance phase=' + str(phase))
        r_e_cut_chank_number = identity_cut_chank(phase)
        # e_cut_chank_number = identity_cut_chank_no_intersections(phase)
        r_e_muscle_chank_freq = identify_muscle_freq(phase, muscle_fibers_freq_hi, muscle_fibers_freq_lo)
        ## print (cut_chank_number)
        ## print (muscle_chank_freq)
        r_e_poisson_cut_input[:].rates = 0*Hz
        for chn in r_e_cut_chank_number:
            ## print (str(chn*cut_chank) + ": " + str((chn+1)*cut_chank-1))
            r_e_poisson_cut_input[chn*cut_chank:(chn+1)*cut_chank-1].rates = 200*Hz
            ## print (poisson_cut_input)
        r_e_poisson_muscle_input.rates = r_e_muscle_chank_freq
        run(dur*ms)
        ## run(dur*ms, report='text')
    
print('Simulation ended ...')

Simulation started ...
step=0
step=1
step=2
step=3
step=4
step=5
step=6
step=7
step=8
step=9
step=10
step=11
step=12
step=13
step=14
step=15
step=16
step=17
step=18
step=19
Simulation ended ...


# Save data

## Left leg

In [11]:
!pwd

/home/talanov/brian2/bypass/out/long_run


In [12]:
save_steps_num = 60
if steps_num < 60:
    save_steps_num = 60
    
save_path = save_path_prefix + str(save_steps_num) + '/'
# https://brian2.readthedocs.io/en/stable/user/recording.html
## Left leg
## Extensor
# Save motor neurons voltages
l_e_motor_neuron_data = l_e_motor_neurons_voltages.get_states(['t', 'v'])
with open(save_path + 'l_e_motor_neurons_voltages.pickle', 'wb') as f:
  pickle.dump(l_e_motor_neuron_data, f)
# Save weights ⬆️ see above 
l_e_cut2rg_data = l_e_cut2rg_stdp_monitor.get_states(['t', 's'])
with open(save_path + 'l_e_cut2rg_weigts.pickle', 'wb') as f:
  pickle.dump(l_e_cut2rg_data, f)

# l_e_rg2InE_data = l_e_rg2InE_stdp_monitor.get_states(['t', 's'])
# with open(save_path + 'l_e_rg2InE_weigts.pickle', 'wb') as f:
#   pickle.dump(l_e_rg2InE_data, f)

# l_f_rg2InE_data = l_f_rg2InE_stdp_monitor.get_states(['t', 's'])
# with open(save_path + 'l_f_rg2InE_weigts.pickle', 'wb') as f:
#   pickle.dump(l_f_rg2InE_data, f)

print('Saved left leg E voltages')

Saved left leg E voltages


In [13]:
## Left leg
## Extensor
# Save spikes 
l_e_motor_neuron_spikes = l_e_motor_neurons_monitor.get_states(['t', 'i'])
with open(save_path + 'l_e_motor_neurons_spikes.pickle', 'wb') as f:
  pickle.dump(l_e_motor_neuron_spikes, f)

l_e_rg_neuron_spikes = l_e_rg_neurons_monitor.get_states(['t', 'i'])
with open(save_path + 'l_e_rg_neurons_spikes.pickle', 'wb') as f:
  pickle.dump(l_e_rg_neuron_spikes, f)

l_e_cut_spikes = l_e_cut_monitor.get_states(['t', 'i'])
with open(save_path + 'l_e_cut_spikes.pickle', 'wb') as f:
  pickle.dump(l_e_cut_spikes, f)

# l_e_bs_spikes = l_e_bs_monitor.get_states(['t', 'i'])
# with open(save_path + 'l_e_bs_spikes.pickle', 'wb') as f:
#   pickle.dump(l_e_bs_spikes, f)

# l_e_Ia_spikes = l_e_Ia_neurons_monitor.get_states(['t', 'i'])
# with open(save_path + 'l_e_Ia_spikes.pickle', 'wb') as f:
#   pickle.dump(l_e_Ia_spikes, f)

l_e_muscle_spikes = l_e_muscle_monitor.get_states(['t', 'i'])
with open(save_path + 'l_e_muscle_spikes.pickle', 'wb') as f:
  pickle.dump(l_e_muscle_spikes, f)

print('Saved left leg E spikes')

Saved left leg E spikes


## Right leg

In [14]:
## Right leg
## Extensor
# Save motor neurons voltages
r_e_motor_neuron_data = r_e_motor_neurons_voltages.get_states(['t', 'v'])
with open(save_path + 'r_e_motor_neurons_voltages.pickle', 'wb') as f:
  pickle.dump(r_e_motor_neuron_data, f)
# Save weights ⬆️ see above 
r_e_cut2rg_data = r_e_cut2rg_stdp_monitor.get_states(['t', 's'])
with open(save_path + 'r_e_cut2rg_weigts.pickle', 'wb') as f:
  pickle.dump(r_e_cut2rg_data, f)

# r_e_rg2InE_data = r_e_rg2InE_stdp_monitor.get_states(['t', 's'])
# with open(save_path + 'r_e_rg2InE_weigts.pickle', 'wb') as f:
#   pickle.dump(r_e_rg2InE_data, f)

# r_f_rg2InE_data = r_f_rg2InE_stdp_monitor.get_states(['t', 's'])
# with open(save_path + 'r_f_rg2InE_weigts.pickle', 'wb') as f:
#   pickle.dump(r_f_rg2InE_data, f)

print('Saved right leg E voltages')

Saved right leg E voltages


In [15]:
## Right leg
## Extensor
# Save spikes 
r_e_motor_neuron_spikes = r_e_motor_neurons_monitor.get_states(['t', 'i'])
with open(save_path + 'r_e_motor_neurons_spikes.pickle', 'wb') as f:
  pickle.dump(r_e_motor_neuron_spikes, f)

r_e_rg_neuron_spikes = r_e_rg_neurons_monitor.get_states(['t', 'i'])
with open(save_path + 'r_e_rg_neurons_spikes.pickle', 'wb') as f:
  pickle.dump(r_e_rg_neuron_spikes, f)

r_e_cut_spikes = r_e_cut_monitor.get_states(['t', 'i'])
with open(save_path + 'r_e_cut_spikes.pickle', 'wb') as f:
  pickle.dump(r_e_cut_spikes, f)

# r_e_bs_spikes = r_e_bs_monitor.get_states(['t', 'i'])
# with open(save_path + 'r_e_bs_spikes.pickle', 'wb') as f:
#   pickle.dump(r_e_bs_spikes, f)

# r_e_Ia_spikes = r_e_Ia_neurons_monitor.get_states(['t', 'i'])
# with open(save_path + 'r_e_Ia_spikes.pickle', 'wb') as f:
#   pickle.dump(r_e_Ia_spikes, f)

r_e_muscle_spikes = r_e_muscle_monitor.get_states(['t', 'i'])
with open(save_path + 'r_e_muscle_spikes.pickle', 'wb') as f:
  pickle.dump(r_e_muscle_spikes, f)
print('Saved right leg E spikes')


Saved right leg E spikes
